# Fine-tuning material amounts

This notebook provides examples of some post-hoc modifications that can be implemented in the database after the initial run, to progressively refine estimations of material amounts.

### Setup

In [ ]:
import psycopg as pg
import pandas as pd

In [ ]:
params='dbname=macrocomponents user=postgres password=mypassword' # Write the parameters to connect to your PostgresQL database here.

In [ ]:
def run_sql (DBparameters,SQLcode):
    try:
        # connect to the PostgreSQL database
        connector = pg.connect(DBparameters)

        # create a new cursor
        cur = connector.cursor()

        # execute the SQL statement
        cur.execute(SQLcode)

        # commit the changes to the database
        connector.commit()

        # close communication with the database
        cur.close()

    except (Exception, pg.DatabaseError) as error:
        print(error)

    finally:
        if connector is not None:
            connector.close()

### Changing the amount of bricks in external walls

The amount of bricks in external walls usually depends on the height of the building. This can be implemented in the model based on the number of floors, instead of using a constant for the thickness of brick walls.

In [ ]:
SEF = 1.2 

In [ ]:
FloorHeight=3.5

In [ ]:
WindowWallRatio=0.2

In [ ]:
get_perimeter_sql=f"SELECT (CASE WHEN (b.byg054AntalEtager IS NULL OR b.byg054AntalEtager = 0) THEN SQRT(b.byg041BebyggetAreal)*2*(%s+1/%s) ELSE SQRT(b.byg038SamletBygningsareal/b.byg054AntalEtager)*2*(%s+1/%s) END) as perimeter" % (SEF,SEF,SEF,SEF) #Rough approximation if the building has storeys of different sizes

In [ ]:
SQL="""
WITH brick_ext_walls_new AS
(SELECT 
    b.id_lokalId bbrid,
    typ.id,
    typ.name,
    perimeter,
    ext_wall_surf,
    brick_thickness,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_ext_walls bmap
    ON b.id_lokalId = bmap.bbr_id
INNER JOIN ext_wall_types typ
    ON typ.id=2
INNER JOIN ext_walls_to_subcomponents submap
    ON typ.id = submap.ext_wall_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (%s) ltp,
LATERAL (SELECT (CASE WHEN b.byg054AntalEtager IS NOT NULL THEN perimeter*b.byg054AntalEtager*%s*(1-%s) ELSE perimeter*%s*(1-%s) END) AS ext_wall_surf) lt1,
LATERAL (SELECT (CASE WHEN b.byg054AntalEtager IS NOT NULL THEN 0.228 * (0.85+(b.byg054AntalEtager+1)/8) ELSE 0.228 END) AS brick_thickness) lt2,
LATERAL (SELECT ext_wall_surf*brick_thickness*pmap.amount/0.336 AS amount_product) lt3
WHERE pr.name NOT LIKE '%%plaster%%'
)

UPDATE results_material_amounts
SET amount = brick_ext_walls_new.amount_product
FROM brick_ext_walls_new
WHERE (results_material_amounts.bbr_id, results_material_amounts.element, results_material_amounts.product)=(brick_ext_walls_new.bbrid, 'ext_wall', brick_ext_walls_new.product) 

""" %(get_perimeter_sql, FloorHeight, WindowWallRatio, FloorHeight, WindowWallRatio)

In [ ]:
run_sql(params,SQL)

Then we can always update the total amounts table:

In [ ]:
update_tot_amounts_sql="""
WITH t AS(
SELECT 
    element,
    SUM(amount) sum,
    unit,
    product
FROM results_material_amounts
GROUP BY (element, unit, product))

UPDATE tot_material_amounts tma
SET amount=t.sum
FROM t
WHERE (tma.element, tma.unit, tma.product) = (t.element, t.unit, t.product)
"""

In [ ]:
run_sql(params,update_tot_amounts_sql)